# Quickstart Guide

We highly advise thoroughly reviewing the [BLPAPI Core Developer Guide](https://data.bloomberglp.com/professional/sites/10/2017/03/BLPAPI-Core-Developer-Guide.pdf) when developing your Bloomberg Queries. This guide is essential for understanding function parameters, request options, overrides, error messages, and troubleshooting methods.

In [1]:
import datetime
import json

import pandas

from blp import blp

In [2]:
bquery = blp.BlpQuery().start()

# Bloomberg Queries

## Data History (BDH)

In [3]:
bquery.bdh(
    ["SPY US Equity", "TLT US Equity"],
    ["PX_LAST", "VOLUME"],
    start_date="20190101",
    end_date="20190110",
    options={"adjustmentSplit": True}
)

,date,security,PX_LAST,VOLUME
0,2019-01-02,SPY US Equity,250.18,126925199.0
1,2019-01-02,TLT US Equity,122.15,19841527.0
2,2019-01-03,SPY US Equity,244.21,144140692.0
3,2019-01-03,TLT US Equity,123.54,21187045.0
4,2019-01-04,SPY US Equity,252.39,142628834.0
5,2019-01-04,TLT US Equity,122.11,12970226.0
6,2019-01-07,SPY US Equity,254.38,103139100.0
7,2019-01-07,TLT US Equity,121.75,8498104.0
8,2019-01-08,SPY US Equity,256.77,102512587.0
9,2019-01-08,TLT US Equity,121.43,7737103.0


## Data Point (BDP)

In [4]:
bquery.bdp(["CL1 Comdty"], ["NAME", "PX_LAST"])

,security,NAME,PX_LAST
0,CL1 Comdty,Generic 1st 'CL' Future,75.27


## Data Set (BDS)

In [5]:
bquery.bds("BCOM Index", "INDX_MWEIGHT")

,Member Ticker and Exchange Code,Percentage Weight
0,BOH4,2.67
1,C H4,4.31
2,CLH4,8.47
3,COH4,7.92
4,CTH4,1.55
5,GCG4,17.52
6,HGH4,5.56
7,HOH4,2.00
8,KCH4,3.74
9,KWH4,1.44


## Intraday Bar (BDIB)

In [6]:
bquery.bdib(
    "AAPL US Equity",
    event_type="TRADE",
    interval=60,
    start_datetime="2023-11-01", # Different date format
    end_datetime="2023-11-02", # Different date format
)

,time,open,high,low,close,volume,numEvents,value
0,2023-11-01 13:30:00,171.000,171.97,170.120,171.828,8272888,41465,1.415716e+09
1,2023-11-01 14:30:00,171.835,172.08,171.000,171.430,5189464,26241,8.903882e+08
2,2023-11-01 15:30:00,171.440,172.00,171.195,171.910,2887219,14505,4.957753e+08
3,2023-11-01 16:30:00,171.905,172.38,171.630,172.162,3109733,14308,5.350173e+08
4,2023-11-01 17:30:00,172.160,172.71,171.910,172.620,4089553,20561,7.043825e+08
5,2023-11-01 18:30:00,172.620,174.22,172.100,173.910,8659156,46255,1.499576e+09
6,2023-11-01 19:30:00,173.920,174.23,173.610,173.970,13746588,32967,2.391479e+09


## Equity Screening (BEQS)

In [7]:
bquery.beqs(
    "Core Capital Ratios",
    screen_type="GLOBAL",
    options={"Group": "General"}
)

,security,Tot Rsk Bsd Cap Rt LF,Ticker,Short Name,Market Cap,Price:D-1,Tier 1 Cap Rt LF,Moody's Issuer Rtg,S&P LT FC Iss Cred Rtg
0,8331 JP,11.580000,8331 JP,CHIBA BANK LTD,5.757796e+09,7.071664,11.510000,A1,A-
1,ABN NA,19.799999,ABN NA,ABN AMRO BANK-CV,1.285043e+10,14.816520,16.500000,Aa3,A
2,ACA FP,17.900000,ACA FP,CREDIT AGRICOLE,4.330626e+10,14.157763,13.700000,Aa3,A+
3,BAC US,15.400000,BAC US,BANK OF AMERICA,2.645561e+11,33.860001,13.600000,A1,A-
4,BBVA SM,16.510000,BBVA SM,BBVA,5.424467e+10,9.074981,14.450000,A3,A
5,BNP FP,17.799999,BNP FP,BNP PARIBAS,7.980755e+10,69.411812,15.550000,Aa3,A+
6,BNS CN,17.200001,BNS CN,BANK OF NOVA SCO,5.818445e+10,47.749039,14.800000,A2,A+
7,CBK GR,19.200001,CBK GR,COMMERZBANK,1.470946e+10,11.836692,16.500000,A2,A-
8,CFR US,15.280000,CFR US,CULLEN/FROST,6.890196e+09,107.989998,13.810000,A3,A-
9,DANSKE DC,23.200001,DANSKE DC,DANSKE BANK A/S,2.261516e+10,26.175419,20.600000,A3,A+


## BQL Queries

`bquery.bql` accepts a Bloomberg Query String. For optimal query creation, utilize Excel's BQL Builder in Advanced View. Once your query is ready, click the "Copy" button to copy it, and then paste it into the `bquery.bql` argument.

In [9]:
result = bquery.bql("get(px_last) for(['IBM US Equity', 'AAPL US Equity'])")
# bquery.bql returns a list of dataframes with one dataframe per field.
result[0]

,field,id,value,DATE,CURRENCY
0,px_last,IBM US Equity,163.210007,2023-12-26T00:00:00Z,USD
1,px_last,AAPL US Equity,193.050003,2023-12-26T00:00:00Z,USD


# Advanced Query Techniques

## Using Overrides

Various fields can be overriden. For info on what fields support what overrides, the best place to check is using `FLDS` from the terminal.

In [10]:
pandas.concat(
    [
        bquery.bdp(["IBM US Equity"], ["CRNCY_ADJ_PX_LAST"]),
        bquery.bdp(
            ["IBM US Equity"], ["CRNCY_ADJ_PX_LAST"], overrides=[("EQY_FUND_CRNCY", "EUR")]
        )
    ],
    axis=1,
    keys=["default", "override"]
)

default                         override                  
        security CRNCY_ADJ_PX_LAST       security CRNCY_ADJ_PX_LAST
0  IBM US Equity            163.21  IBM US Equity          147.9558

In [11]:
bquery.bds(
    "DAX Index",
    "INDX_MWEIGHT_HIST",
    overrides=[("END_DATE_OVERRIDE", "20230630")],
)

,Index Member,Percent Weight
0,1COV GY,-2.424536e-14
1,ADS GY,-2.424536e-14
2,AIR GY,-2.424536e-14
3,ALV GY,-2.424536e-14
4,BAS GY,-2.424536e-14
5,BAYN GY,-2.424536e-14
6,BEI GY,-2.424536e-14
7,BMW GY,-2.424536e-14
8,BNR GY,-2.424536e-14
9,CBK GY,-2.424536e-14


## Using SEDOLs

In [12]:
bquery.bdp(
    ["SPY US Equity", "TLT US Equity"],
    ["ID_SEDOL1"],
)

,security,ID_SEDOL1
0,SPY US Equity,2840215
1,TLT US Equity,2971546


In [13]:
bquery.bdh(
    ["/sedol1/2840215", "/sedol1/2971546"],
    ["PX_LAST", "VOLUME"],
    start_date="20190101",
    end_date="20190110",
)

,date,security,PX_LAST,VOLUME
0,2019-01-02,/sedol1/2840215,230.5557,126925199.0
1,2019-01-02,/sedol1/2971546,109.3493,19841527.0
2,2019-01-03,/sedol1/2840215,225.0540,144140692.0
3,2019-01-03,/sedol1/2971546,110.5936,21187045.0
4,2019-01-04,/sedol1/2840215,232.5924,142628834.0
5,2019-01-04,/sedol1/2971546,109.3135,12970226.0
6,2019-01-07,/sedol1/2840215,234.4263,103139100.0
7,2019-01-07,/sedol1/2971546,108.9912,8498104.0
8,2019-01-08,/sedol1/2840215,236.6288,102512587.0
9,2019-01-08,/sedol1/2971546,108.7048,7737103.0


# Troubleshooting

The most frequent errors in `blpapi` are due to poor input. As `blp` is a wrapper for `blpapi`, these errors are passed to the user. To resolve them, examine the error message for issues, then contact the Bloomberg help desk.

For instance, the code below will produce an error because it includes an invalid ticker.

```python
bquery.bdh(
    ["NOT_A_TICKER US Equity", "SPY US Equity"],
    ["PX_LAST", "VOLUME"],
    start_date="20190101",
    end_date="20190110",
)
```
```
TypeError: Response for 'NOT_A_TICKER US Equity' contains securityError
{
    "securityError": {
        "source": "3923::bbdbh4",
        "code": 15,
        "category": "BAD_SEC",
        "message": "Unknown/Invalid securityInvalid Security [nid:3923] ",
        "subcategory": "INVALID_SECURITY",
    }
}
```

To ignore errors, instantiate a parser as shown below:

In [14]:
bquery = blp.BlpQuery(parser=blp.BlpParser(raise_security_errors=False)).start()

bquery.bdh(
    ["NOT_A_TICKER US Equity", "SPY US Equity"],
    ["PX_LAST", "VOLUME"],
    start_date="20190101",
    end_date="20190110",
)

Response for 'NOT_A_TICKER US Equity' contains securityError {'securityError': {'source': '53359:rsfhdsvc2', 'code': 15, 'category': 'BAD_SEC', 'message': 'Unknown/Invalid securityInvalid Security [nid:53359]', 'subcategory': 'INVALID_SECURITY'}}


,date,security,PX_LAST,VOLUME
0,2019-01-02,SPY US Equity,230.5557,126925199.0
1,2019-01-03,SPY US Equity,225.0540,144140692.0
2,2019-01-04,SPY US Equity,232.5924,142628834.0
3,2019-01-07,SPY US Equity,234.4263,103139100.0
4,2019-01-08,SPY US Equity,236.6288,102512587.0
5,2019-01-09,SPY US Equity,237.7347,95006554.0
6,2019-01-10,SPY US Equity,238.5733,96823923.0


# Additional Features

## Retrieve JSON Data

### Query Generation Utility Methods

In [15]:
query = blp.create_query(
    request_type="HistoricalDataRequest",
    values={
        "securities": ["SPY US Equity"],
        "fields": ["VOLUME"],
        "startDate": "20190101",
        "endDate": "20190105",
    },
)
print(json.dumps(query, indent=2))

{
  "HistoricalDataRequest": {
    "securities": [
      "SPY US Equity"
    ],
    "fields": [
      "VOLUME"
    ],
    "startDate": "20190101",
    "endDate": "20190105"
  }
}


In [16]:
# deal with Timestamps for pretty printing for response

def ts_to_json(obj):
    return obj.strftime("%Y-%m-%dT%H:%M:%S.%f%z")


def time_to_json(obj):
    return obj.strftime("%H:%M:%S")

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, pandas.Timestamp):
            return ts_to_json(obj)
        if isinstance(obj, datetime.time):
            return time_to_json(obj)
        return super().default(obj)

In [17]:
resp = bquery.query(query, parse=False, collector=list)
print(json.dumps(resp, indent=2, cls=CustomJSONEncoder))

[
  {
    "eventType": 5,
    "eventTypeName": "blpapi.Event.RESPONSE",
    "messageNumber": 0,
    "message": {
      "fragmentType": 0,
      "correlationIds": [
        34
      ],
      "messageType": "HistoricalDataResponse",
      "timeReceived": "2023-12-27T02:21:35.053315+0000",
      "element": {
        "HistoricalDataResponse": {
          "securityData": {
            "security": "SPY US Equity",
            "eidData": [],
            "sequenceNumber": 0,
            "fieldExceptions": [],
            "fieldData": [
              {
                "fieldData": {
                  "date": "2019-01-02T00:00:00.000000",
                  "VOLUME": 126925199.0
                }
              },
              {
                "fieldData": {
                  "date": "2019-01-03T00:00:00.000000",
                  "VOLUME": 144140692.0
                }
              },
              {
                "fieldData": {
                  "date": "2019-01-04T00:00:00.000000",
       

## Support for Context Manager

In [18]:
with blp.BlpQuery() as bq:
    df = bq.bdh(
        ["GME US Equity"],
        ["PX_LAST", "VOLUME"],
        start_date="20210101",
        end_date="20210130",
    )
df

,date,security,PX_LAST,VOLUME
0,2021-01-04,GME US Equity,4.3125,40089896.0
1,2021-01-05,GME US Equity,4.3425,19845828.0
2,2021-01-06,GME US Equity,4.5900,24224992.0
3,2021-01-07,GME US Equity,4.5200,24517104.0
4,2021-01-08,GME US Equity,4.4225,25927840.0
5,2021-01-11,GME US Equity,4.9850,59710448.0
6,2021-01-12,GME US Equity,4.9875,28242660.0
7,2021-01-13,GME US Equity,7.8500,578006944.0
8,2021-01-14,GME US Equity,9.9775,374869640.0
9,2021-01-15,GME US Equity,8.8750,187465432.0


# Streaming Data

In [19]:
with blp.BlpStream() as bs:
    bs.subscribe({"USDCAD Curncy": {"fields": ["LAST_PRICE"]}})
    n = 0
    for ev in bs.events(timeout=60):
        print(json.dumps(ev, indent=2, cls=CustomJSONEncoder))
        n += 1
        if n > 1:
            break

SESSION_STATUS - Message 0 - SessionConnectionDown = {
    server = "localhost:8194"
}



{
  "fragmentType": 1,
  "correlationIds": [
    "USDCAD Curncy"
  ],
  "messageType": "MarketDataEvents",
  "timeReceived": "2023-12-27T02:21:44.066568+0000",
  "element": {
    "MarketDataEvents": {
      "MKTDATA_EVENT_TYPE": "SUMMARY",
      "MKTDATA_EVENT_SUBTYPE": "INITPAINT",
      "BID": 1.3198,
      "ASK": 1.3199,
      "BEST_BID": 1.3198,
      "BEST_ASK": 1.3199,
      "BID_ALL_SESSION": 1.3198,
      "ASK_ALL_SESSION": 1.3199,
      "IND_BID_FLAG": false,
      "IND_ASK_FLAG": false,
      "BID_TDY": 1.3198,
      "PRICE_LAST_BID_RT": 1.3198,
      "ASK_TDY": 1.3199,
      "PRICE_LAST_ASK_RT": 1.3199,
      "BID_DIR": 1,
      "ASK_DIR": -1,
      "BID2": 1.3198,
      "ASK2": 1.3199,
      "BID_ALL_SESSION_TDY_RT": 1.3198,
      "ASK_ALL_SESSION_TDY_RT": 1.3199,
      "CURRENT_SESSION_RT": 2,
      "PX_OPEN_ALL_WITH_SWITCHOVER_RT": 1.3195171356201172,
      "PX_CLOSE_ALL_WITH_SWITCHOVER_RT": 1.3195,
      "LAST_PRICE_ALL_SESSION_DIR_RT": 1,
      "SIMP_LAST_PX_ALL_SESS_DI